# CS5340 Lecture 2:  k-Means, GMMs,and Expectation Maximization#


Lecturer: Harold Soh (harold@comp.nus.edu.sg)

Graduate TAs: Abdul Fatir Ansari and Chen Kaiqi (AY19/20)

This notebook is a supplement to Lecture 7 of CS5340: Uncertainty Modeling in AI


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal


In [ ]:
# computes the elucidean distances from each point X to each point in C
def euclidean(X, c):
    c = c.reshape(1, X.shape[1])
    return np.linalg.norm(X - c, ord=2, axis=1)

In [ ]:
# let's create some data!
# you can play aorund with this!
dataset = 'varied' # 'simple', 'varied' or 'anisotropic'
n_samples = 1000

# set random seed for consistency
seed = 170
np.random.seed(seed)

if dataset == 'simple':
    X, y = make_blobs(n_samples=n_samples, random_state=seed, centers=3)
elif dataset == 'varied':
    # blobs with varied variances
    X,y = make_blobs(n_samples=n_samples,cluster_std=[1.0, 2.5, 0.5],random_state=seed)
elif dataset == 'anisotropic':
    X, y = make_blobs(n_samples=n_samples, random_state=seed, centers=3)
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    X = np.dot(X, transformation)
else:
    print('No such dataset! Defaulting to easy')
    X, y = make_blobs(n_samples=n_samples, random_state=seed, centers=3)

plt.scatter(X[:, 0], X[:, 1], s=3)
plt.title('Data')
plt.show()

# KMeans

* Set the number of clusters $k$ and randomly initialize the clusters centers $\{\mu_i\}_{i=1}^k$.

In [ ]:
# set random seed for consistency
seed = 1
np.random.seed(seed)

k = 3  # Number of clusters
d = X.shape[1]  # Dimensionality of data points
labels = np.arange(k, dtype=np.uint)
cx = np.random.uniform(low=X[:, 0].min(), high=X[:, 0].max(), size=k)[:, None]
cy = np.random.uniform(low=X[:, 1].min(), high=X[:, 1].max(), size=k)[:, None]
centers = np.hstack([cx, cy])

### Assignment Step
* Assign each datapoint to the nearest cluster center.

### Update Step
* Update each cluster center to the mean of the points assigned to it. 

In [ ]:
itr = 0
max_iter = 100
plot_freq = 10

while True and itr < max_iter:
    
    # Assignment step
    distances = np.asarray([euclidean(X, c) for c in centers]) # k x n array of distances
    assignments = np.argmin(distances, 0) # list of size 'n', cluster indices for each datapoint
    
    # Plot current assignments
    if itr%plot_freq == 0:
        plt.scatter(X[:, 0], X[:, 1], c=assignments, s=3)
        plt.scatter(centers[:, 0], centers[:, 1], marker='X', color='r')
        plt.title('Step {:d}'.format(itr+1))
        plt.show()
    
    # Update step
    centers = np.asarray([np.mean(X[assignments == l], 0) for l in labels]) # k x d array of cluster centers
    
    # check converged
    if itr > 0:
        if (prev_assgn == assignments).all():
            print("Converged at Step %d!"%(itr))
            break
        
    prev_assgn = assignments
    
    itr += 1
    
plt.scatter(X[:, 0], X[:, 1], c=assignments, s=3)
plt.scatter(centers[:, 0], centers[:, 1], marker='X', color='r')
plt.title('Step {:d}'.format(itr+1))
plt.show()

# GMM

The likelihood of a GMM is given by
$$
p(x_1, \dots, x_n|\theta) = \prod_{i=1}^n\sum_{j=1}^k \pi_k\mathcal{N}(x_i;\mu_j,\Sigma_j)
$$

* Intialize $\pi_k, \mu_k, \Sigma_k$.
* **E-Step**: Evaluate responsibilities $\gamma(z_{nk})$ where $\gamma(z_{nk}) = p(z_{nk}=1|x_n)$.
* **M-Step**: Assign new values to $\pi_k, \mu_k, \Sigma_k$ by maximizing the log-likelihood w.r.t. each parameter.

In [ ]:
np.random.seed(1)
k = 3
pis = np.ones(k)/(k)
rgb = np.array([np.array((1., 0., 0.)), np.array((0., 1., 0.)), np.array((0., 0., 1.)), np.array((0., 0., 0.))])
cx = np.random.uniform(low=X[:, 0].min(), high=X[:, 0].max(), size=len(pis))[:, None]
cy = np.random.uniform(low=X[:, 1].min(), high=X[:, 1].max(), size=len(pis))[:, None]
mus = np.hstack([cx, cy])
Sigmas = np.asarray([np.eye(X.shape[1]) for l in pis])

In [ ]:
def Estep(pis, mus, Sigmas):
    # get the responsibilities
    pi_N = np.asarray([pis[i] * multivariate_normal.pdf(X, mus[i], Sigmas[i]) for i in range(k)]) # k x n
    resp = pi_N / np.sum(pi_N, 0, keepdims=True) # k x n
    return resp, pi_N

def Mstep(resp, pis, mus, Sigmas):
    mus = np.sum(resp[:, :, None] * X[None, :, :], axis=1)/np.sum(resp, axis=1, keepdims=True)
    S = X[None, :, :, None] - mus[:, None, :, None] # k x n x d x 1
    S_ST = np.matmul(S, np.transpose(S, axes=(0, 1, 3, 2))) # k x n x d x d
    Sigmas = np.sum(S_ST * resp[:, :, None, None], axis=1)
    Sigmas = Sigmas / (np.sum(resp, axis=1).reshape(k, 1, 1))
    pis = np.sum(resp, axis=1) / np.sum(resp)
    return pis, mus, Sigmas

In [ ]:
prev_ll = -np.inf
itr = 0
max_itr = 1000
plot_freq = 1
while True and itr < max_itr:
    # E-Step
    resp, pi_N = Estep(pis, mus, Sigmas) 
    
    # Compute log-likelihood and check for convergence
    log_ll = np.log(1e-8 + np.sum(pi_N, 0)).sum()
    if log_ll - prev_ll < 0.1:
        print('Converged at step %d'%(itr))
        break
            
    # Plot results
    if itr % plot_freq == 0:
        # Assign a color to each datapoint
        colors = np.sum(resp[:, None, :] * rgb[:k, :, None], 0).T
        plt.scatter(X[:, 0], X[:, 1], c=colors, s=3)
        plt.scatter(mus[:, 0], mus[:, 1], marker='X', edgecolors='k')
        plt.title('Step %d : log-likelihood: %.2f'%(itr, log_ll))
        plt.show()
    
    # M-Step
    pis, mus, Sigmas = Mstep(resp, pis, mus, Sigmas)
    
    prev_ll = log_ll
    itr += 1
    
#final plot
colors = np.sum(resp[:, None, :] * rgb[:k, :, None], 0).T
plt.scatter(X[:, 0], X[:, 1], c=colors, s=3)
plt.scatter(mus[:, 0], mus[:, 1], marker='X', edgecolors='k')
plt.title('Step %d : log-likelihood: %.2f'%(itr, log_ll))
plt.show()

# Singularity Issues

In [ ]:
seed = 7813
np.random.seed(seed)
k = 4
pis = np.ones(k)/(k)
cx = np.random.uniform(low=X[:, 0].min(), high=X[:, 0].max(), size=len(pis))[:, None]
cy = np.random.uniform(low=X[:, 1].min(), high=X[:, 1].max(), size=len(pis))[:, None]
mus = np.hstack([cx, cy])
Sigmas = np.asarray([np.eye(X.shape[1]) for l in pis])

In [ ]:
prev_ll = -np.inf
itr = 0
max_itr = 1000
plot_freq = 10
while True and itr < max_itr:
    # E-Step
    # get the responsibilities
    resp, pi_N = Estep(pis, mus, Sigmas) 
    
    # Compute log-likelihood and check for convergence
    log_ll = np.log(1e-8 + np.sum(pi_N, 0)).sum()
    if log_ll - prev_ll < 0.1:
        print('Converged at step %d'%(itr))
        break
            
    # Plot results
    if itr % plot_freq == 0:
        # Assign a color to each datapoint
        colors = np.sum(resp[:, None, :] * rgb[:k, :, None], 0).T
        plt.scatter(X[:, 0], X[:, 1], c=colors, s=3)
        plt.scatter(mus[:, 0], mus[:, 1], marker='X', edgecolors='k')
        plt.title('Step %d : log-likelihood: %.2f'%(itr, log_ll))
        plt.show()
    
    # M-Step
    pis, mus, Sigmas = Mstep(resp, pis, mus, Sigmas)
    
    prev_ll = log_ll
    itr += 1
    
#final plot
colors = np.sum(resp[:, None, :] * rgb[:k, :, None], 0).T
plt.scatter(X[:, 0], X[:, 1], c=colors, s=3)
plt.scatter(mus[:, 0], mus[:, 1], marker='X', edgecolors='k')
plt.title('Step %d : log-likelihood: %.2f'%(itr, log_ll))
plt.show()

Singularity issues can be solved by adding a prior to $\Sigma_k$. The MLE equation for $\Sigma_k$ is given by

$$
N_k\Sigma_k - \sum_{i=1}^n(x_n - \mu_k)(x_n - \mu_k)^\top = 0
$$

Let's set the prior on $\Sigma_k$ to be the Inverse Wishart. The MAP equation is now given by

$$
N_k\Sigma_k - \sum_{i=1}^n(x_n - \mu_k)(x_n - \mu_k)^\top + \frac{\partial}{\partial \Sigma^{-1}_k}\left[\mathcal{W}^{-1}(\Sigma_k; \Psi, \nu)\right] = 0
$$

where 
$$
\mathcal{W}^{-1}(\Sigma; \Psi, \nu) = \frac{|\Psi|^{\nu/2}}{2^{\nu p/2}\Gamma_p(\nu/2)}|\Sigma|^{-(\nu + p + 1)/2}\exp\left(-\frac{1}{2}\mathrm{tr}(\Psi\Sigma^{-1})\right)
$$

In [ ]:
# you can take the code on adding the prior on your own. 